<a href="https://colab.research.google.com/github/27Roger/Data-Science/blob/main/Github_IDS_Project1.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [107]:
from scipy.stats import mannwhitneyu, ttest_ind, kstest, f_oneway
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np

In [57]:
df = pd.read_csv("movieReplicationSet.csv")
df.head()

,The Life of David Gale (2003),Wing Commander (1999),Django Unchained (2012),Alien (1979),Indiana Jones and the Last Crusade (1989),Snatch (2000),Rambo: First Blood Part II (1985),Fargo (1996),Let the Right One In (2008),Black Swan (2010),...,When watching a movie I cheer or shout or talk or curse at the screen,When watching a movie I feel like the things on the screen are happening to me,As a movie unfolds I start to have problems keeping track of events that happened earlier,"The emotions on the screen ""rub off"" on me - for instance if something sad is happening I get sad or if something frightening is happening I get scared",When watching a movie I get completely immersed in the alternative reality of the film,Movies change my position on social economic or political issues,When watching movies things get so intense that I have to stop watching,Gender identity (1 = female; 2 = male; 3 = self-described),Are you an only child? (1: Yes; 0: No; -1: Did not respond),Movies are best enjoyed alone (1: Yes; 0: No; -1: Did not respond)
0,NaN,NaN,4.0,NaN,3.0,NaN,NaN,NaN,NaN,NaN,...,1.0,6.0,2.0,5.0,5.0,5.0,1.0,1.0,0,1
1,NaN,NaN,1.5,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,3.0,1.0,1.0,6.0,5.0,3.0,2.0,1.0,0,0
2,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,5.0,4.0,3.0,5.0,5.0,4.0,4.0,1.0,1,0
3,NaN,NaN,2.0,NaN,3.0,NaN,NaN,NaN,NaN,4.0,...,3.0,1.0,1.0,4.0,5.0,3.0,1.0,1.0,0,1
4,NaN,NaN,3.5,NaN,0.5,NaN,0.5,1.0,NaN,0.0,...,2.0,3.0,2.0,5.0,6.0,4.0,4.0,1.0,1,1


In [58]:
df.shape

(1097, 477)

In [170]:
#Function definitions
def get_category_difference_t(movie, x, y):
  c1 = x[movie].values
  c1 = c1[np.isfinite(c1)]
  c2 = y[movie].values
  c2 = c2[np.isfinite(c2)]
  #t, tp = ttest_ind(c1, c2)
  U, up = mannwhitneyu(c1, c2, alternative="two-sided")
  return(U, up)

def get_category_difference_g(movie, x, y):
  c1 = x[movie].values
  c1 = c1[np.isfinite(c1)]
  c2 = y[movie].values
  c2 = c2[np.isfinite(c2)]
  #t, tp = ttest_ind(c1, c2)
  U, up = mannwhitneyu(c1, c2, alternative="greater")
  return(U,up)

def get_difference(x, y):
  c1 = x.values
  c1 = c1[np.isfinite(c1)]
  #print(len(c1))
  c2 = y.values
  c2 = c2[np.isfinite(c2)]
  #print(len(c2))
  #t, tp = ttest_ind(c1, c2)
  k, p = kstest(c1, c2, alternative="two-sided")
  return(k,p)

## 1. Popularity Movies - 
Are movies that are more popular (operationalized as having more ratings) rated higher than movies that are less popular? 
[Hint: You can do a median-split of popularity to determine high vs. low popularity movies]   

In [154]:
movies = df.columns.to_list()[:400]
not_null=np.empty((1,400))
pop_median=[]
unpop_median=[]

for i in range(400):
    one_movie=pd.to_numeric(df[movies[i]],errors='coerce').values
    one_movie=one_movie[np.isfinite(one_movie)]
    not_null[0][i]=len(one_movie)
pop_median_det=np.median(not_null)

not_popular_movies=[]
popular_movies=[]
for i in range(400):
  if not_null[0][i]>int(pop_median_det):
    popular_movies.append(movies[i])
  else:
    not_popular_movies.append(movies[i])

for i in range(200):
  one_movie=pd.to_numeric(df[not_popular_movies[i]],errors='coerce').values
  one_movie=one_movie[np.isfinite(one_movie)]
  unpop_median.append(np.median(one_movie))

for i in range(len(popular_movies)):
  one_movie=pd.to_numeric(df[popular_movies[i]],errors='coerce').values
  one_movie=one_movie[np.isfinite(one_movie)]
  pop_median.append(np.median(one_movie))

U, p = mannwhitneyu(pop_median, unpop_median, alternative="greater")
print("U value =", U)
print("\nP value =", p)

U value = 33427.5

P value = 9.929258851707232e-35


## 2. New vs Old Movies 
Are movies that are newer rated differently than movies that are older? [Hint: Do a median split of year of 
release to contrast movies in terms of whether they are old or new] 

In [155]:
movies_list = df.iloc[:,:400]
_date = []
for column in movies_list.columns:
    _date.append((column, int(column[-5:-1])))
sort_date = sorted(_date, key=lambda x:x[1])
old = sort_date[:200]
new = sort_date[200:]
old_movies_list = [x[0] for x in old]
new_movies_list = [x[0] for x in new]
x=df[new_movies_list].median()
y=df[old_movies_list].median()
U, p = mannwhitneyu(x, y, alternative="two-sided")
print("U value =", U)
print("\nP value =", p)

U value = 21937.5

P value = 0.07752243118272252


## 3. Shrek Viewers
Is enjoyment of ‘Shrek (2001)’ gendered, i.e. do male and female viewers rate it differently? 

In [157]:
gender_column = "Gender identity (1 = female; 2 = male; 3 = self-described)"
male_data =  df[df[gender_column]==2].iloc[:,:400]
female_data =  df[df[gender_column]==1].iloc[:,:400]
U, up = get_category_difference_t("Shrek (2001)", male_data, female_data)
print("Mann Whitney U Test U value =", U)
print("Mann Whitney U Test P value =", up)

Mann Whitney U Test U value = 82232.5
Mann Whitney U Test P value = 0.050536625925559006


## 4. Proportion of Movies - Gender Effect
What proportion of movies are rated differently by male and female viewers?

In [161]:
movies = df.iloc[:,:400]
columns = movies.columns
print(columns)
count = 0
for i in columns:
  p1,p2 = get_category_difference_t(i, male_data, female_data)
  if p2 <=0.005:
    # print(column, p)
    count+=1
print(count, "/",len(columns))
print("Proportion :", count/ len(columns))

Index(['The Life of David Gale (2003)', 'Wing Commander (1999)',
       'Django Unchained (2012)', 'Alien (1979)',
       'Indiana Jones and the Last Crusade (1989)', 'Snatch (2000)',
       'Rambo: First Blood Part II (1985)', 'Fargo (1996)',
       'Let the Right One In (2008)', 'Black Swan (2010)',
       ...
       'X-Men 2 (2003)', 'The Usual Suspects (1995)', 'The Mask (1994)',
       'Jaws (1975)', 'Harry Potter and the Chamber of Secrets (2002)',
       'Patton (1970)', 'Anaconda (1997)', 'Twister (1996)',
       'MacArthur (1977)', 'Look Who's Talking (1989)'],
      dtype='object', length=400)
50 / 400
Proportion : 0.125


## 5. Lion King - Viewers 
Do people who are only children enjoy ‘The Lion King (1994)’ more than people with siblings? 

In [158]:
only_child_column = "Are you an only child? (1: Yes; 0: No; -1: Did not respond)"
sibling_data =  df[df[only_child_column]==0].iloc[:,:400]
onlychild_data =  df[df[only_child_column]==1].iloc[:,:400]
U,up = get_category_difference_g("The Lion King (1994)", onlychild_data, sibling_data)
print("Mann Whitney U Test U value =", U)
print("Mann Whitney U Test P value =", up)

Mann Whitney U Test U value = 52929.0
Mann Whitney U Test P value = 0.978419092554931


## 6. Proportion of Movies - Only Child Effect 
What proportion of movies exhibit an “only child effect”, i.e. are rated different by viewers with siblings 
vs. those without? 

In [162]:
movies = df.iloc[:,:400]
columns = movies.columns
count = 0
for column in columns:
  p1,p2 = get_category_difference_t(column, onlychild_data, sibling_data)
  if p2<=0.005:
    # print(column, p)
    count+=1
print(count, "/",len(columns))
print("Proportion :", count/ len(columns))

7 / 400
Proportion : 0.0175


## 7. The Wolf of Wall Street - Viewers
Do people who like to watch movies socially enjoy ‘The Wolf of Wall Street (2013)’ more than those who 
prefer to watch them alone? 

In [159]:
socially_column = "Movies are best enjoyed alone (1: Yes; 0: No; -1: Did not respond)"
social_data =  df[df[socially_column]==0].iloc[:,:400]
alone_data =  df[df[socially_column]==1].iloc[:,:400]
U,up = get_category_difference_g("The Wolf of Wall Street (2013)", social_data, alone_data )
print("Mann Whitney U Test U value =", U)
print("Mann Whitney U Test P value =", up)

Mann Whitney U Test U value = 49303.5
Mann Whitney U Test P value = 0.9436657996253056


## 8. Proportion of Movies - Social Watching Effect 
What proportion of movies exhibit such a “social watching” effect?

In [164]:
movies = df.iloc[:,:400]
columns = movies.columns
count = 0
for i in columns:
  p1,p2 = get_category_difference_t(i, social_data, alone_data)
  if p2<=0.005:
    # print(column, p)
    count+=1
print(count, "/",len(columns))
print("Proportion :", count/ len(columns))

10 / 400
Proportion : 0.025


## 9. Home Alone and Finding Nemo
Is the ratings distribution of ‘Home Alone (1990)’ different than that of ‘Finding Nemo (2003)’?  

In [169]:
home_alone = df["Home Alone (1990)"]
finding_nemo = df["Finding Nemo (2003)"]
f,p = get_difference(home_alone, finding_nemo)
print("KS Test K Value=",f,"P value =", p)

857
1014
KS Test K Value= 0.15269080020897632 P value = 6.379381467525036e-10


## 10. Consistency of Franchises 
There are ratings on movies from several franchises ([‘Star Wars’, ‘Harry Potter’, ‘The Matrix’, ‘Indiana 
Jones’, ‘Jurassic Park’, ‘Pirates of the Caribbean’, ‘Toy Story’, ‘Batman’]) in this dataset. How many of these 
are of inconsistent quality, as experienced by viewers? [Hint: You can use the keywords in quotation marks 
featured in this question to identify the movies that are part of each franchise]  

In [118]:
#To Print the list of all franchise movies in the given movieReplication.csv
'''df = pd.read_csv("movieReplicationSet.csv")
Movies_list = ["Star Wars", "Harry Potter", "The Matrix", "Indiana Jones", "Jurassic Park", "Pirates of the Caribbean", "Toy Story", "Batman"]
franch = {}
for i in Movies_list:
  franch[i] = []
  print(i)
  for c in df.columns:
    if i in c:
      franch[i].append(c)
      print(c)
'''

'df = pd.read_csv("movieReplicationSet.csv")\nMovies_list = ["Star Wars", "Harry Potter", "The Matrix", "Indiana Jones", "Jurassic Park", "Pirates of the Caribbean", "Toy Story", "Batman"]\nfranch = {}\nfor i in Movies_list:\n  franch[i] = []\n  print(i)\n  for c in df.columns:\n    if i in c:\n      franch[i].append(c)\n      print(c)\n'

#STAR WARS

In [134]:
from scipy.stats.mstats_basic import kruskal
df=pd.read_csv('movieReplicationSet.csv',skipinitialspace=True)
listOfMovies=df.columns.to_list()[:400]
keyWord="Star Wars"
sequels=[]
for i in range(400):
    if keyWord in listOfMovies[i]:
        sequels.append(listOfMovies[i])
for i in sequels:
  print(i)
S4=pd.to_numeric(df[sequels[0]],errors='coerce').values
S4 = S4[np.isfinite(S4)]
S2=pd.to_numeric(df[sequels[1]],errors='coerce').values
S2 = S2[np.isfinite(S2)]
S5=pd.to_numeric(df[sequels[2]],errors='coerce').values
S5 = S5[np.isfinite(S5)]
S1=pd.to_numeric(df[sequels[3]],errors='coerce').values
S1 = S1[np.isfinite(S1)]
S6=pd.to_numeric(df[sequels[4]],errors='coerce').values
S6 = S6[np.isfinite(S6)]
S7=pd.to_numeric(df[sequels[5]],errors='coerce').values
S7 = S7[np.isfinite(S7)]

combinedData = np.transpose(np.array([S1,S2,S4,S5,S6,S7], dtype=object))
print("\nMann Whitney Test to check between sequels")
for i in range(len(sequels)-1):
    u,p = mannwhitneyu(combinedData[i],combinedData[i+1])
    
    print("U-value:",u,"P-value:",p)
    
print("\nSignificant difference seen between Star Wars: Episode II - Attack of the Clones (2002) and Star Wars: Episode IV - A New Hope (1977)")

f,p1 = kruskal(S1,S2,S4,S5,S6,S7)
print("\nKruskal-Wallis H-test test to test if population median of all of the groups are equal")
print("F-value:",f,"P-value:",p1)

Star Wars: Episode IV - A New Hope (1977)
Star Wars: Episode II - Attack of the Clones (2002)
Star Wars: Episode V - The Empire Strikes Back (1980)
Star Wars: Episode 1 - The Phantom Menace (1999)
Star Wars: Episode VII - The Force Awakens (2015)
Star Wars: Episode VI - The Return of the Jedi (1983)

Mann Whitney Test to check between sequels
U-value: 124054.5 P-value: 0.4503093559063731
U-value: 101449.0 P-value: 2.7865432291796803e-18
U-value: 125332.0 P-value: 0.08189846600030656
U-value: 116173.0 P-value: 0.5732544284765428
U-value: 116247.0 P-value: 0.11405028669275127

Significant difference seen between Star Wars: Episode II - Attack of the Clones (2002) and Star Wars: Episode IV - A New Hope (1977)

Kruskal-Wallis H-test test to test if population median of all of the groups are equal
F-value: 230.5841753686405 P-value: 8.01647736660335e-48


#Harry Potter

In [126]:
from scipy.stats.mstats_basic import kruskal
df=pd.read_csv('movieReplicationSet.csv',skipinitialspace=True)
listOfMovies=df.columns.to_list()[:400]
keyWord="Harry Potter"
sequels=[]
for i in range(400):
    if keyWord in listOfMovies[i]:
        sequels.append(listOfMovies[i])
for i in sequels:
  print(i)
S4=pd.to_numeric(df[sequels[0]],errors='coerce').values
S4 = S4[np.isfinite(S4)]
S2=pd.to_numeric(df[sequels[1]],errors='coerce').values
S2 = S2[np.isfinite(S2)]
S5=pd.to_numeric(df[sequels[2]],errors='coerce').values
S5 = S5[np.isfinite(S5)]
S1=pd.to_numeric(df[sequels[3]],errors='coerce').values
S1 = S1[np.isfinite(S1)]


combinedData = np.transpose(np.array([S4,S1,S5,S2], dtype=object))
print("\nMann Whitney Test to check between sequels")
for i in range(len(sequels)-1):
    u,p = mannwhitneyu(combinedData[i],combinedData[i+1])
    
    print("U-value:",u,"P-value:",p)
    
print("\nNo significance seen")

f,p1 = kruskal(S1,S2,S4,S5)
print("\nKruskal-Wallis H-test test to test if population median of all of the groups are equal")
print("F-value:",f,"P-value:",p1)

Harry Potter and the Sorcerer's Stone (2001)
Harry Potter and the Deathly Hallows: Part 2 (2011)
Harry Potter and the Goblet of Fire (2005)
Harry Potter and the Chamber of Secrets (2002)

Mann Whitney Test to check between sequels
U-value: 384178.0 P-value: 0.09816683002933
U-value: 342001.0 P-value: 0.8040524609201452
U-value: 331566.0 P-value: 0.4973063104455706

No significance seen

Kruskal-Wallis H-test test to test if population median of all of the groups are equal
F-value: 3.3312307328888706 P-value: 0.34331950837316666


#The Matrix

In [133]:
from scipy.stats.mstats_basic import kruskal
df=pd.read_csv('movieReplicationSet.csv',skipinitialspace=True)
listOfMovies=df.columns.to_list()[:400]
keyWord="The Matrix"
sequels=[]
for i in range(400):
    if keyWord in listOfMovies[i]:
        sequels.append(listOfMovies[i])
for i in sequels:
  print(i)
S4=pd.to_numeric(df[sequels[0]],errors='coerce').values
S4 = S4[np.isfinite(S4)]
S2=pd.to_numeric(df[sequels[1]],errors='coerce').values
S2 = S2[np.isfinite(S2)]
S5=pd.to_numeric(df[sequels[2]],errors='coerce').values
S5 = S5[np.isfinite(S5)]


combinedData = np.transpose(np.array([S5,S2,S4], dtype=object))
print("\nMann Whitney Test to check between sequels")
for i in range(len(sequels)-1):
    u,p = mannwhitneyu(combinedData[i],combinedData[i+1])
    
    print("U-value:",u,"P-value:",p)
    
print("\nSignificant difference seen between The Matrix (1999) and The Matrix Reloaded (2003)")

f,p1 = kruskal(S2,S4,S5)
print("\nKruskal-Wallis H-test test to test if population median of all of the groups are equal")
print("F-value:",f,"P-value:",p1)

The Matrix Revolutions (2003)
The Matrix Reloaded (2003)
The Matrix (1999)

Mann Whitney Test to check between sequels
U-value: 100837.5 P-value: 2.1376654433729527e-07
U-value: 65932.5 P-value: 0.24981112383243775

Significant difference seen between The Matrix (1999) and The Matrix Reloaded (2003)

Kruskal-Wallis H-test test to test if population median of all of the groups are equal
F-value: 48.378866521305774 P-value: 3.1236517880781424e-11


#Indiana Jones

In [137]:
from scipy.stats.mstats_basic import kruskal
df=pd.read_csv('movieReplicationSet.csv',skipinitialspace=True)
listOfMovies=df.columns.to_list()[:400]
keyWord="Indiana Jones"
sequels=[]
for i in range(400):
    if keyWord in listOfMovies[i]:
        sequels.append(listOfMovies[i])
for i in sequels:
  print(i)
S4=pd.to_numeric(df[sequels[0]],errors='coerce').values
S4 = S4[np.isfinite(S4)]
S2=pd.to_numeric(df[sequels[1]],errors='coerce').values
S2 = S2[np.isfinite(S2)]
S5=pd.to_numeric(df[sequels[2]],errors='coerce').values
S5 = S5[np.isfinite(S5)]
S1=pd.to_numeric(df[sequels[3]],errors='coerce').values
S1 = S1[np.isfinite(S1)]


combinedData = np.transpose(np.array([S5,S2,S4,S1], dtype=object))
print("\nMann Whitney Test to check between sequels")
for i in range(len(sequels)-1):
    u,p = mannwhitneyu(combinedData[i],combinedData[i+1])
    
    print("U-value:",u,"P-value:",p)
    
print("\nSignificant difference seen between Indiana Jones and the Raiders of the Lost Ark (1981) and Indiana Jones and the Temple of Doom (1984)")
print("\nSignificant difference seen between Indiana Jones and the Last Crusade (1989) and Indiana Jones and the Kingdom of the Crystal Skull (2008)")

f,p1 = kruskal(S2,S4,S5)
print("\nKruskal-Wallis H-test test to test if population median of all of the groups are equal")
print("F-value:",f,"P-value:",p1)

Indiana Jones and the Last Crusade (1989)
Indiana Jones and the Temple of Doom (1984)
Indiana Jones and the Raiders of the Lost Ark (1981)
Indiana Jones and the Kingdom of the Crystal Skull (2008)

Mann Whitney Test to check between sequels
U-value: 97240.0 P-value: 1.8017248346687264e-05
U-value: 107144.0 P-value: 0.3070842227239434
U-value: 120933.0 P-value: 0.00017791395232999937

Significant difference seen between Indiana Jones and the Raiders of the Lost Ark (1981) and Indiana Jones and the Temple of Doom (1984)

Significant difference seen between Indiana Jones and the Last Crusade (1989) and Indiana Jones and the Kingdom of the Crystal Skull (2008)

Kruskal-Wallis H-test test to test if population median of all of the groups are equal
F-value: 19.92433047366296 P-value: 4.715053336633116e-05


#Jurassic Park

In [139]:
from scipy.stats.mstats_basic import kruskal
df=pd.read_csv('movieReplicationSet.csv',skipinitialspace=True)
listOfMovies=df.columns.to_list()[:400]
keyWord="Jurassic Park"
sequels=[]
for i in range(400):
    if keyWord in listOfMovies[i]:
        sequels.append(listOfMovies[i])
for i in sequels:
  print(i)
S4=pd.to_numeric(df[sequels[0]],errors='coerce').values
S4 = S4[np.isfinite(S4)]
S2=pd.to_numeric(df[sequels[1]],errors='coerce').values
S2 = S2[np.isfinite(S2)]
S5=pd.to_numeric(df[sequels[2]],errors='coerce').values
S5 = S5[np.isfinite(S5)]

combinedData = np.transpose(np.array([S5,S4,S2], dtype=object))
print("\nMann Whitney Test to check between sequels")
for i in range(len(sequels)-1):
    u,p = mannwhitneyu(combinedData[i],combinedData[i+1])
    
    print("U-value:",u,"P-value:",p)
    
print("\nSignificant difference seen between The Lost World: Jurassic Park (1997) and Jurassic Park (1993)")
print("\nSignificant difference seen between Jurassic Park (1993) and Jurassic Park III (2001)")

f,p1 = kruskal(S2,S4,S5)
print("\nKruskal-Wallis H-test test to test if population median of all of the groups are equal")
print("F-value:",f,"P-value:",p1)

The Lost World: Jurassic Park (1997)
Jurassic Park III (2001)
Jurassic Park (1993)

Mann Whitney Test to check between sequels
U-value: 188684.0 P-value: 0.0006738642151062385
U-value: 169371.5 P-value: 0.00027537558821442277

Significant difference seen between The Lost World: Jurassic Park (1997) and Jurassic Park (1993)

Significant difference seen between Jurassic Park (1993) and Jurassic Park III (2001)

Kruskal-Wallis H-test test to test if population median of all of the groups are equal
F-value: 46.59088064385298 P-value: 7.636930084362221e-11


#Pirates of the Caribbean

In [141]:
from scipy.stats.mstats_basic import kruskal
df=pd.read_csv('movieReplicationSet.csv',skipinitialspace=True)
listOfMovies=df.columns.to_list()[:400]
keyWord="Pirates of the Caribbean"
sequels=[]
for i in range(400):
    if keyWord in listOfMovies[i]:
        sequels.append(listOfMovies[i])
for i in sequels:
  print(i)
S4=pd.to_numeric(df[sequels[0]],errors='coerce').values
S4 = S4[np.isfinite(S4)]
S2=pd.to_numeric(df[sequels[1]],errors='coerce').values
S2 = S2[np.isfinite(S2)]
S5=pd.to_numeric(df[sequels[2]],errors='coerce').values
S5 = S5[np.isfinite(S5)]

combinedData = np.transpose(np.array([S5,S4,S2], dtype=object))
print("\nMann Whitney Test to check between sequels")
for i in range(len(sequels)-1):
    u,p = mannwhitneyu(combinedData[i],combinedData[i+1])
    
    print("U-value:",u,"P-value:",p)
    
print("\nSignificant difference seen between Pirates of the Caribbean: Dead Man's Chest (2006) and Pirates of the Caribbean: The Curse of the Black Pearl (2003)")

f,p1 = kruskal(S2,S4,S5)
print("\nKruskal-Wallis H-test test to test if population median of all of the groups are equal")
print("F-value:",f,"P-value:",p1)

Pirates of the Caribbean: Dead Man's Chest (2006)
Pirates of the Caribbean: At World's End (2007)
Pirates of the Caribbean: The Curse of the Black Pearl (2003)

Mann Whitney Test to check between sequels
U-value: 308667.0 P-value: 8.817372076227087e-06
U-value: 258329.5 P-value: 0.25147758276137844

Significant difference seen between Pirates of the Caribbean: Dead Man's Chest (2006) and Pirates of the Caribbean: The Curse of the Black Pearl (2003)

Kruskal-Wallis H-test test to test if population median of all of the groups are equal
F-value: 20.64399756002606 P-value: 3.2901287079094474e-05


#Toy Story

In [143]:
from scipy.stats.mstats_basic import kruskal
df=pd.read_csv('movieReplicationSet.csv',skipinitialspace=True)
listOfMovies=df.columns.to_list()[:400]
keyWord="Toy Story"
sequels=[]
for i in range(400):
    if keyWord in listOfMovies[i]:
        sequels.append(listOfMovies[i])
for i in sequels:
  print(i)
S4=pd.to_numeric(df[sequels[0]],errors='coerce').values
S4 = S4[np.isfinite(S4)]
S2=pd.to_numeric(df[sequels[1]],errors='coerce').values
S2 = S2[np.isfinite(S2)]
S5=pd.to_numeric(df[sequels[2]],errors='coerce').values
S5 = S5[np.isfinite(S5)]

combinedData = np.transpose(np.array([S5,S4,S2], dtype=object))
print("\nMann Whitney Test to check between sequels")
for i in range(len(sequels)-1):
    u,p = mannwhitneyu(combinedData[i],combinedData[i+1])
    
    print("U-value:",u,"P-value:",p)
    
print("\nSignificant difference seen between Toy Story 3 (2010) and Toy Story 2 (1999)")
print("\nSignificant difference seen between Toy Story (1995) and Toy Story 2 (1999)")

f,p1 = kruskal(S2,S4,S5)
print("\nKruskal-Wallis H-test test to test if population median of all of the groups are equal")
print("F-value:",f,"P-value:",p1)

Toy Story 2 (1999)
Toy Story 3 (2010)
Toy Story (1995)

Mann Whitney Test to check between sequels
U-value: 472908.0 P-value: 6.329730946125958e-06
U-value: 356610.0 P-value: 8.408513827408602e-05

Significant difference seen between Toy Story 3 (2010) and Toy Story 2 (1999)

Significant difference seen between Toy Story (1995) and Toy Story 2 (1999)

Kruskal-Wallis H-test test to test if population median of all of the groups are equal
F-value: 24.38599493626327 P-value: 5.065805156537524e-06


#Batman

In [145]:
from scipy.stats.mstats_basic import kruskal
df=pd.read_csv('movieReplicationSet.csv',skipinitialspace=True)
listOfMovies=df.columns.to_list()[:400]
keyWord="Batman"
sequels=[]
for i in range(400):
    if keyWord in listOfMovies[i]:
        sequels.append(listOfMovies[i])
for i in sequels:
  print(i)
S4=pd.to_numeric(df[sequels[0]],errors='coerce').values
S4 = S4[np.isfinite(S4)]
S2=pd.to_numeric(df[sequels[1]],errors='coerce').values
S2 = S2[np.isfinite(S2)]
S5=pd.to_numeric(df[sequels[2]],errors='coerce').values
S5 = S5[np.isfinite(S5)]

combinedData = np.transpose(np.array([S4,S2,S5], dtype=object))
print("\nMann Whitney Test to check between sequels")
for i in range(len(sequels)-1):
    u,p = mannwhitneyu(combinedData[i],combinedData[i+1])
    
    print("U-value:",u,"P-value:",p)
    
print("\nSignificant difference seen between Batman & Robin (1997) and Batman (1989)")
print("\nSignificant difference seen between Batman (1989) and Batman: The Dark Knight (2008)")

f,p1 = kruskal(S2,S4,S5)
print("\nKruskal-Wallis H-test test to test if population median of all of the groups are equal")
print("F-value:",f,"P-value:",p1)

Batman & Robin (1997)
Batman (1989)
Batman: The Dark Knight (2008)

Mann Whitney Test to check between sequels
U-value: 48397.5 P-value: 1.5631433682224575e-09
U-value: 99593.0 P-value: 2.5183727470948456e-15

Significant difference seen between Batman & Robin (1997) and Batman (1989)

Significant difference seen between Batman (1989) and Batman: The Dark Knight (2008)

Kruskal-Wallis H-test test to test if population median of all of the groups are equal
F-value: 190.53496872634642 P-value: 4.2252969509030006e-42


#EXTRA CREDIT

Do people who have trouble following a movie rate Chicago (2002) differently?
Ho= People don't rate Chicago (2002) based on whether they followed the story of the movie

In [183]:
df=pd.read_csv('movieReplicationSet.csv',skipinitialspace=True)
trouble_columns  = "I have trouble following the story of a movie"
trouble_data =  df[df[trouble_columns]<=df[trouble_columns].median()].iloc[:,:400]
not_troubled_data =  df[df[trouble_columns]>df[trouble_columns].median()].iloc[:,:400]
_,p = get_category_difference_g("Chicago (2002)", trouble_data, not_troubled_data)
print("P value =",p)

P value = 0.0031912618750724302


Proportion of Movies rated by viewers who exhibit the personality that they have trouble following the story of a movie.

In [182]:
mov_columns = movies.columns
count = 0
for i in mov_columns:
  _,p = get_category_difference_g(i, trouble_data, not_troubled_data)
  if p<=0.005:
    # print(column, p)
    print(i)
    count+=1
print(count, "/",len(columns))
print("Proportion :", count/len(columns))

Indiana Jones and the Last Crusade (1989)
Psycho (1960)
Schindler's List (1993)
The Godfather: Part II (1974)
Indiana Jones and the Raiders of the Lost Ark (1981)
Fight Club (1999)
12 Monkeys (1995)
Full Metal Jacket (1987)
Leon (1994)
Girl Interrupted (1999)
Life is Beautiful (1997)
Saving Private Ryan (1998)
The Lord of the Rings: The Two Towers (2002)
Forrest Gump (1994)
Goodfellas (1990)
The Prestige (2006)
Batman: The Dark Knight (2008)
Donnie Darko (2001)
Requiem for a Dream (2000)
Chicago (2002)
A Clockwork Orange (1971)
Reservoir Dogs (1992)
Memento (2000)
American Psycho (2000)
Who Framed Roger Rabbit (1988)
American History X (1998)
Pirates of the Caribbean: The Curse of the Black Pearl (2003)
Men in Black (1997)
Eternal Sunshine of the Spotless Mind (2004)
Ocean's Eleven (2001)
One Flew Over the Cuckoo's Nest (1975)
31 / 400
Proportion : 0.0775
